# Extracting Transcripts from YouTube videos

In [1]:
import numpy

In [2]:
import urllib.request
from bs4 import BeautifulSoup

URL = []

textToSearch = 'canon camera review'                 ## Entering the text to be searhed on the youtube
query = urllib.parse.quote(textToSearch)             ## parsing it to urllib for url extraction

url = "https://www.youtube.com/results?search_query=" + query
response = urllib.request.urlopen(url)
html = response.read()

soup = BeautifulSoup(html, 'html.parser')
for vid in soup.findAll(attrs={'class':'yt-uix-tile-link'}):
    
    #print('https://www.youtube.com' + vid['href'])
    URL.append('https://www.youtube.com' + vid['href'])  ## appending all the urls on all the pages in a single list


In [3]:
from urllib.parse import urlparse

def video_id(url):      ## function to extract ids for each videop for its transcript exrtraction, if they have one
    
    o = urlparse(url)
    if o.netloc == 'youtu.be':
        return o.path[1:]
    
    elif o.netloc in ('www.youtube.com', 'youtube.com'):
        if o.path == '/watch':
            id_index = o.query.index('v=')
            return o.query[id_index+2:id_index+13]
        elif o.path[:7] == '/embed/':
            return o.path.split('/')[2]
        elif o.path[:3] == '/v/':
            return o.path.split('/')[2]
    
    return None  # fail?

In [6]:
video_ids = []

for url in URL:
    video_ids.append(video_id(url))      ## a list of extracted video ids on the urls

In [11]:
from youtube_transcript_api import YouTubeTranscriptApi

Transcripts = []

for ids in video_ids:
    
    try:
        transcript = YouTubeTranscriptApi.get_transcript(ids)  ## extracting transcripts of each id if it has one on the youtube page
    except:
        continue
    
    Transcripts.append(transcript)

In [13]:
print(len(Transcripts))             ## out of the many videos only 14 had trascripts given by the youtube

14


In [26]:
import os

os.makedirs(textToSearch)

In [49]:
## all that is left is to extract the tyext from these transcript dictionaries

for i in range(len(Transcripts)):                      ## for each video
    
    f = open(textToSearch + '/' + textToSearch + str(i), "w")
    transcript_text = []
    
    for j in range(len(Transcripts[i])):               ## for each line in transcript
        
        text = Transcripts[i][j]['text'] + '\n'        ## we only need text for review analysis
        transcript_text.append(text)
    
    transcript_text = "".join(transcript_text)
    f.write(transcript_text)
    
f.close()